In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sentence_transformers import SentenceTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import LinearSVC
from tensorflow import keras

In [2]:
df = pd.read_csv("export/final_data.csv")
df.head()

,id,job_title,location,salary_currency,career_level,experience_level,education_level,employment_type,job_function,job_benefits,company_process_time,company_size,company_industry,job_description,salary,job_description_cleaned,annotations,skills
0,1,Facility Maintenance & Smart Warehouse Manager,Bandung,IDR,Manajer/Asisten Manajer,5 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,"Manufaktur,Pemeliharaan",NaN,NaN,NaN,NaN,Deskripsi PekerjaanRequirements :D3/SI from re...,NaN,deskripsi pekerjaanrequirements si from reputa...,{'text': 'deskripsi pekerjaanrequirements si f...,"['electrical inspection', 'management system',..."
1,2,Procurement Department Head,Jakarta Raya,IDR,Manajer/Asisten Manajer,5 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",Penuh Waktu,"Manufaktur,Pembelian/Manajemen Material",NaN,25 days,51 - 200 pekerja,Manajemen/Konsulting HR,Job Role: 1. Responsible for material availabi...,NaN,job role responsible for material availabili...,{'text': 'job role responsible for material av...,"['heavy equipment', 'contract management', 'pr..."
2,3,SALES ADMIN,Jakarta Barat,IDR,Supervisor/Koordinator,4 tahun,Sarjana (S1),Penuh Waktu,"Penjualan / Pemasaran,Penjualan Ritel","Waktu regular, Senin - Jumat;Bisnis (contoh: K...",30 days,51 - 200 pekerja,Umum & Grosir,Internal Sales & AdminJob Description :We are ...,NaN,internal sales adminjob description we are loo...,{'text': 'internal sales adminjob description ...,"['microsoft office', 'heat exchanger', 'carbon..."
3,4,City Operation Lead Shopee Express (Cirebon),Cirebon,IDR,Supervisor/Koordinator,5 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",Penuh Waktu,"Pelayanan,Logistik/Rantai Pasokan","Tip;Waktu regular, Senin - Jumat;Kasual (conto...",21 days,2001 - 5000 pekerja,Retail/Merchandise,Job Description:Responsible for HSE implementa...,NaN,job description responsible for hse implementa...,{'text': 'job description responsible for hse ...,"['operation management', 'analytical skill', '..."
4,5,Japanese Interpreter,Bekasi,IDR,Pegawai (non-manajemen & non-supervisor),2 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,"Lainnya,Jurnalis/Editor",NaN,23 days,201 - 500 pekerja,Manajemen/Konsulting HR,Overview: Our clients is manufacture for autom...,NaN,overview our clients is manufacture for automo...,{'text': 'overview our clients is manufacture ...,"['japanese', 'translator', 'english', 'non', '..."


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17182 entries, 0 to 17181
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       17182 non-null  int64  
 1   job_title                17182 non-null  object 
 2   location                 17182 non-null  object 
 3   salary_currency          17178 non-null  object 
 4   career_level             17182 non-null  object 
 5   experience_level         17182 non-null  object 
 6   education_level          17182 non-null  object 
 7   employment_type          17182 non-null  object 
 8   job_function             17182 non-null  object 
 9   job_benefits             14440 non-null  object 
 10  company_process_time     13377 non-null  object 
 11  company_size             15488 non-null  object 
 12  company_industry         16412 non-null  object 
 13  job_description          17182 non-null  object 
 14  salary                

In [5]:
# Get a NumPy array of all unique values
unique_values_list = df['job_function'].unique()
print(len(unique_values_list))

68


In [16]:
job_function_mapping = {

    # Finance & Accounting
    "Akuntansi / Keuangan,Audit & Pajak": "Finance & Accounting",
    "Akuntansi / Keuangan,Akuntansi Umum / Pembiayaan": "Finance & Accounting",
    "Akuntansi / Keuangan,Perbankan / Jasa Finansial ": "Finance & Accounting",
    "Akuntansi / Keuangan,Keuangan / Investasi Perusahaan ": "Finance & Accounting",
    "Sains,Aktuaria/Statistik": "Finance & Accounting",

    # IT & Software
    "Komputer/Teknologi Informasi,IT-Perangkat Lunak": "IT & Software",
    "Komputer/Teknologi Informasi,IT-Admin Jaringan/Sistem/Database": "IT & Software",
    "Komputer/Teknologi Informasi,IT-Perangkat Keras": "IT & Software",

    # Engineering (Non-IT)
    "Teknik,Teknik Lingkungan": "Engineering",
    "Teknik,Teknik Lainnya": "Engineering",
    "Teknik,Mekanikal": "Engineering",
    "Teknik,Teknik Elektronika": "Engineering",
    "Teknik,Teknik Elektro": "Engineering",
    "Teknik,Teknik Industri": "Engineering",
    "Teknik,Teknik Kimia": "Engineering",
    "Bangunan/Konstruksi,Teknik Sipil/Konstruksi Bangunan": "Engineering",
    "Bangunan/Konstruksi,Survei Kuantitas": "Engineering",

    # Manufacturing & Operations
    "Manufaktur,Manufaktur": "Manufacturing & Operations",
    "Manufaktur,Pemeliharaan": "Manufacturing & Operations",
    "Manufaktur,Pembelian/Manajemen Material": "Manufacturing & Operations",
    "Manufaktur,Penjaminan Kualitas / QA": "Manufacturing & Operations",
    "Pelayanan,Logistik/Rantai Pasokan": "Manufacturing & Operations",

    # Sales & Marketing
    "Penjualan / Pemasaran,Pemasaran/Pengembangan Bisnis": "Sales & Marketing",
    "Penjualan / Pemasaran,Digital Marketing": "Sales & Marketing",
    "Penjualan / Pemasaran,Penjualan - Jasa Keuangan": "Sales & Marketing",
    "Penjualan / Pemasaran,Penjualan - Teknik/Teknikal/IT": "Sales & Marketing",
    "Penjualan / Pemasaran,Penjualan Ritel": "Sales & Marketing",
    "Penjualan / Pemasaran,Merchandising": "Sales & Marketing",
    "Penjualan / Pemasaran,Penjualan - Korporasi": "Sales & Marketing",
    "Penjualan / Pemasaran,Telesales/Telemarketing": "Sales & Marketing",
    "Penjualan / Pemasaran,E-commerce": "Sales & Marketing",
    "Seni/Media/Komunikasi,Periklanan": "Sales & Marketing",

    # HR & Administration
    "Sumber Daya Manusia/Personalia,Sumber Daya Manusia / HR": "HR & Administration",
    "Sumber Daya Manusia/Personalia,Staf / Administrasi umum": "HR & Administration",
    "Sumber Daya Manusia/Personalia,Sekretaris": "HR & Administration",
    "Sumber Daya Manusia/Personalia,Top Management / Manajemen Tingkat Atas": "HR & Administration",

    # Creative & Media
    "Seni/Media/Komunikasi,Seni / Desain Kreatif": "Creative & Media",
    "Seni/Media/Komunikasi,Hubungan Masyarakat": "Creative & Media",
    "Seni/Media/Komunikasi,Hiburan": "Creative & Media",
    "Lainnya,Jurnalis/Editor": "Creative & Media",

    # Healthcare & Life Sciences
    "Layanan Kesehatan,Diagnosa/Lainnya": "Healthcare",
    "Layanan Kesehatan,Farmasi": "Healthcare",
    "Layanan Kesehatan,Praktisi/Asisten Medis": "Healthcare",
    "Sains,Bioteknologi": "Healthcare",
    "Sains,Biomedis": "Healthcare",
    "Sains,Kimia": "Healthcare",
    "Sains,Geologi/Geofisika": "Healthcare",

    # Education & Training
    "Pendidikan/Pelatihan,Pendidikan": "Education",
    "Pendidikan/Pelatihan,Pelatihan & Pengembangan": "Education",

    # Hospitality & Service
    "Hotel/Restoran,Makanan/Minuman/Pelayanan Restoran": "Hospitality & Service",
    "Hotel/Restoran,Hotel/Pariwisata": "Hospitality & Service",
    "Pelayanan,Layanan Pelanggan": "Hospitality & Service",
    "Pelayanan,Teknikal & Bantuan Pelanggan": "Hospitality & Service",
    "Pelayanan,Perawatan Pribadi": "Hospitality & Service",

    # Legal & Public Service
    "Pelayanan,Pengacara / Asisten Legal": "Legal & Public Service",
    "Pelayanan,Angkatan Bersenjata": "Legal & Public Service",

    # Science & Agriculture
    "Sains,Pertanian": "Science & Agriculture",
    "Sains,Teknologi Makanan/Ahli Gizi": "Science & Agriculture",
    "Sains,Sains & Teknologi": "Science & Agriculture",

    # Other
    "Bangunan/Konstruksi,Arsitek/Desain Interior": "Other",
    "Bangunan/Konstruksi,Properti/Real Estate": "Other",
    "Lainnya,Pekerjaan Umum": "Other",
    "Lainnya,Lainnya/Kategori tidak tersedia": "Other",
}

df["job_function_group"] = (
    df["job_function"]
    .map(job_function_mapping)
    .fillna("Other")
)

In [17]:
encoders_dict = {}

columns_to_encode = ['job_function_group']
for col in columns_to_encode:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    encoders_dict[col] = le

df.head()

,id,job_title,location,salary_currency,career_level,experience_level,education_level,employment_type,job_function,job_benefits,company_process_time,company_size,company_industry,job_description,salary,job_description_cleaned,annotations,skills,job_function_group
0,1,Facility Maintenance & Smart Warehouse Manager,Bandung,IDR,Manajer/Asisten Manajer,5 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,"Manufaktur,Pemeliharaan",NaN,NaN,NaN,NaN,Deskripsi PekerjaanRequirements :D3/SI from re...,NaN,deskripsi pekerjaanrequirements si from reputa...,{'text': 'deskripsi pekerjaanrequirements si f...,"['electrical inspection', 'management system',...",9
1,2,Procurement Department Head,Jakarta Raya,IDR,Manajer/Asisten Manajer,5 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",Penuh Waktu,"Manufaktur,Pembelian/Manajemen Material",NaN,25 days,51 - 200 pekerja,Manajemen/Konsulting HR,Job Role: 1. Responsible for material availabi...,NaN,job role responsible for material availabili...,{'text': 'job role responsible for material av...,"['heavy equipment', 'contract management', 'pr...",9
2,3,SALES ADMIN,Jakarta Barat,IDR,Supervisor/Koordinator,4 tahun,Sarjana (S1),Penuh Waktu,"Penjualan / Pemasaran,Penjualan Ritel","Waktu regular, Senin - Jumat;Bisnis (contoh: K...",30 days,51 - 200 pekerja,Umum & Grosir,Internal Sales & AdminJob Description :We are ...,NaN,internal sales adminjob description we are loo...,{'text': 'internal sales adminjob description ...,"['microsoft office', 'heat exchanger', 'carbon...",11
3,4,City Operation Lead Shopee Express (Cirebon),Cirebon,IDR,Supervisor/Koordinator,5 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",Penuh Waktu,"Pelayanan,Logistik/Rantai Pasokan","Tip;Waktu regular, Senin - Jumat;Kasual (conto...",21 days,2001 - 5000 pekerja,Retail/Merchandise,Job Description:Responsible for HSE implementa...,NaN,job description responsible for hse implementa...,{'text': 'job description responsible for hse ...,"['operation management', 'analytical skill', '...",9
4,5,Japanese Interpreter,Bekasi,IDR,Pegawai (non-manajemen & non-supervisor),2 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,"Lainnya,Jurnalis/Editor",NaN,23 days,201 - 500 pekerja,Manajemen/Konsulting HR,Overview: Our clients is manufacture for autom...,NaN,overview our clients is manufacture for automo...,{'text': 'overview our clients is manufacture ...,"['japanese', 'translator', 'english', 'non', '...",0


In [12]:
y = df["job_function_group"].fillna("unknown")
print("n_classes:", y.nunique())
print(y.value_counts())

n_classes: 13
job_function_group
Sales & Marketing             5424
IT & Software                 2895
Finance & Accounting          1929
Manufacturing & Operations    1638
HR & Administration           1269
Engineering                   1181
Creative & Media               714
Other                          599
Hospitality & Service          587
Education                      438
Legal & Public Service         207
Healthcare                     180
Science & Agriculture          121
Name: count, dtype: int64


In [20]:
merge_map = {
    12: 2,  # Science & Agriculture → Engineering
    8:  4,  # Legal & Public Service → HR & Administration
    5:  6   # Healthcare → Hospitality & Service
}

y = y.replace(merge_map)

In [21]:
# y = df["job_function_group"].fillna("unknown")
print("n_classes:", y.nunique())
print(y.value_counts())

n_classes: 10
job_function_group
11    5424
7     2895
3     1929
9     1638
4     1476
2     1302
6      767
0      714
10     599
1      438
Name: count, dtype: int64


In [6]:
# TARGET = 500
# dfs = []

# for label, group in df.groupby("job_function_group"):
#     if len(group) > TARGET:
#         # Downsample
#         sampled = group.sample(TARGET, random_state=42)
#     else:
#         # Upsample
#         sampled = group.sample(TARGET, replace=True, random_state=42)

#     dfs.append(sampled)

# df_balanced = pd.concat(dfs).sample(frac=1, random_state=42)  # shuffle

In [22]:
tfidf = TfidfVectorizer(
    max_features=1000,        # penting agar RF tidak overload
    ngram_range=(1, 2),
    # stop_words="english",
    min_df=2,
    max_df=0.9,
    sublinear_tf=True
)

In [23]:
df["role_text"] = df["job_description"] + " - " + df['experience_level'] + " - " + df["education_level"]

In [24]:
df.head()

,id,job_title,location,salary_currency,career_level,experience_level,education_level,employment_type,job_function,job_benefits,company_process_time,company_size,company_industry,job_description,salary,job_description_cleaned,annotations,skills,job_function_group,role_text
0,1,Facility Maintenance & Smart Warehouse Manager,Bandung,IDR,Manajer/Asisten Manajer,5 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,"Manufaktur,Pemeliharaan",NaN,NaN,NaN,NaN,Deskripsi PekerjaanRequirements :D3/SI from re...,NaN,deskripsi pekerjaanrequirements si from reputa...,{'text': 'deskripsi pekerjaanrequirements si f...,"['electrical inspection', 'management system',...",9,Deskripsi PekerjaanRequirements :D3/SI from re...
1,2,Procurement Department Head,Jakarta Raya,IDR,Manajer/Asisten Manajer,5 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",Penuh Waktu,"Manufaktur,Pembelian/Manajemen Material",NaN,25 days,51 - 200 pekerja,Manajemen/Konsulting HR,Job Role: 1. Responsible for material availabi...,NaN,job role responsible for material availabili...,{'text': 'job role responsible for material av...,"['heavy equipment', 'contract management', 'pr...",9,Job Role: 1. Responsible for material availabi...
2,3,SALES ADMIN,Jakarta Barat,IDR,Supervisor/Koordinator,4 tahun,Sarjana (S1),Penuh Waktu,"Penjualan / Pemasaran,Penjualan Ritel","Waktu regular, Senin - Jumat;Bisnis (contoh: K...",30 days,51 - 200 pekerja,Umum & Grosir,Internal Sales & AdminJob Description :We are ...,NaN,internal sales adminjob description we are loo...,{'text': 'internal sales adminjob description ...,"['microsoft office', 'heat exchanger', 'carbon...",11,Internal Sales & AdminJob Description :We are ...
3,4,City Operation Lead Shopee Express (Cirebon),Cirebon,IDR,Supervisor/Koordinator,5 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",Penuh Waktu,"Pelayanan,Logistik/Rantai Pasokan","Tip;Waktu regular, Senin - Jumat;Kasual (conto...",21 days,2001 - 5000 pekerja,Retail/Merchandise,Job Description:Responsible for HSE implementa...,NaN,job description responsible for hse implementa...,{'text': 'job description responsible for hse ...,"['operation management', 'analytical skill', '...",9,Job Description:Responsible for HSE implementa...
4,5,Japanese Interpreter,Bekasi,IDR,Pegawai (non-manajemen & non-supervisor),2 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,"Lainnya,Jurnalis/Editor",NaN,23 days,201 - 500 pekerja,Manajemen/Konsulting HR,Overview: Our clients is manufacture for autom...,NaN,overview our clients is manufacture for automo...,{'text': 'overview our clients is manufacture ...,"['japanese', 'translator', 'english', 'non', '...",0,Overview: Our clients is manufacture for autom...


In [25]:
X_text = df["role_text"].astype(str)
y = df["job_function_group"].astype(int)

num_classes = y.nunique()

In [26]:
X_train, X_test, y_train, y_test = train_test_split(
    X_text,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [27]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

MAX_VOCAB = 30000
MAX_LEN = 200

tokenizer = Tokenizer(
    num_words=MAX_VOCAB,
    oov_token="<OOV>"
)

tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq  = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding="post")
X_test_pad  = pad_sequences(X_test_seq,  maxlen=MAX_LEN, padding="post")

In [28]:
import tensorflow as tf
from tensorflow.keras import layers, models

EMBED_DIM = 256

inputs = layers.Input(shape=(MAX_LEN,))

x = layers.Embedding(
    input_dim=MAX_VOCAB,
    output_dim=EMBED_DIM,
    embeddings_regularizer=tf.keras.regularizers.l2(1e-6)
)(inputs)

# 🔥 Stacked BiLSTM
x = layers.Bidirectional(
    layers.LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)
)(x)

x = layers.Bidirectional(
    layers.LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)
)(x)

# 🔥 Self-Attention
x = layers.Attention()([x, x])

# 🔥 Dual Pooling
avg_pool = layers.GlobalAveragePooling1D()(x)
max_pool = layers.GlobalMaxPooling1D()(x)
x = layers.Concatenate()([avg_pool, max_pool])

# 🔥 Classification Head
x = layers.BatchNormalization()(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.5)(x)

x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.4)(x)

outputs = layers.Dense(num_classes, activation="softmax")(x)

model = models.Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 200)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 200, 256)  │  7,680,000 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 200, 256)  │    394,240 │ embedding[0][0]   │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_1     │ (None, 200, 128)  │    164,352 │ bidirectional[0]… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention           │ (None, 200, 128)  │          0 │ bidirectional_1[… │
│ (Attention)         │                   │            │ bidirectional_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 128)       │          0 │ attention[0][0]   │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 128)       │          0 │ attention[0][0]   │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 256)       │          0 │ global_average_p… │
│ (Concatenate)       │                   │            │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 256)       │      1,024 │ concatenate[0][0] │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │     65,792 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 256)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │     32,896 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 128)       │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 13)        │      1,677 │ dropout_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 8,339,981 (31.81 MB)

 Trainable params: 8,339,469 (31.81 MB)

 Non-trainable params: 512 (2.00 KB)

In [29]:
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=3,
    restore_best_weights=True
)

history = model.fit(
    X_train_pad,
    y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=64,
    callbacks=[early_stop]
)

Epoch 1/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 195s 1s/step - accuracy: 0.2737 - loss: 2.2548 - val_accuracy: 0.5449 - val_loss: 2.2190
Epoch 2/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 188s 1s/step - accuracy: 0.5974 - loss: 1.3492 - val_accuracy: 0.7235 - val_loss: 1.6986
Epoch 3/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 200s 1s/step - accuracy: 0.7494 - loss: 0.8855 - val_accuracy: 0.7937 - val_loss: 0.9713
Epoch 4/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 219s 1s/step - accuracy: 0.8230 - loss: 0.6245 - val_accuracy: 0.7945 - val_loss: 0.7724
Epoch 5/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 296s 2s/step - accuracy: 0.8844 - loss: 0.4408 - val_accuracy: 0.8156 - val_loss: 0.7839
Epoch 6/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 260s 2s/step - accuracy: 0.9020 - loss: 0.3510 - val_accuracy: 0.8199 - val_loss: 0.8087
Epoch 7/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 240s 1s/step - accuracy: 0.9238 - loss: 0.2700 - val_accuracy: 0.8105 - val_loss: 0.8952


In [30]:
test_loss, test_acc = model.evaluate(X_test_pad, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

108/108 ━━━━━━━━━━━━━━━━━━━━ 17s 155ms/step - accuracy: 0.7974 - loss: 0.7805
Test Accuracy: 0.7972


In [31]:
y_pred = model.predict(X_test_pad)
y_pred_label = np.argmax(y_pred, axis=1)

print(classification_report(y_test, y_pred_label))

108/108 ━━━━━━━━━━━━━━━━━━━━ 21s 190ms/step
              precision    recall  f1-score   support

           0       0.65      0.57      0.61       143
           1       0.78      0.72      0.75        88
           2       0.70      0.72      0.71       236
           3       0.95      0.83      0.89       386
           4       0.81      0.67      0.73       254
           5       0.00      0.00      0.00        36
           6       0.67      0.63      0.65       117
           7       0.94      0.92      0.93       579
           8       0.97      0.68      0.80        41
           9       0.68      0.73      0.70       328
          10       0.67      0.27      0.38       120
          11       0.78      0.95      0.85      1085
          12       0.00      0.00      0.00        24

    accuracy                           0.80      3437
   macro avg       0.66      0.59      0.62      3437
weighted avg       0.79      0.80      0.78      3437



/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [52]:
def predict_job_function(texts):
    seq = tokenizer.texts_to_sequences(texts)
    # print(seq)
    pad = pad_sequences(seq, maxlen=MAX_LEN, padding="post")
    print(pad)
    probs = model.predict(pad)
    labels = probs.argmax(axis=1)
    return labels, probs

In [54]:
texts = [
    """Saya memiliki pengalaman bekerja di bidang kreatif dan media, khususnya dalam pembuatan konten visual dan digital. Saya terbiasa mengelola media sosial, membuat desain grafis, serta mengedit foto dan video untuk kebutuhan branding dan promosi. Saya memahami konsep storytelling, visual identity, dan tren media digital. Selain itu, saya sering berkolaborasi dengan tim marketing untuk menghasilkan konten yang relevan dengan target audiens dan mendukung tujuan kampanye perusahaan."""
]

labels, probs = predict_job_function(texts)
print(labels)

[[9644   60   86   90   21  149  659    7   87 1590   47 1004  597  648
     7  113 9644 1065  618   87  527  109  628 1876  183 4418 1616    7
   431   31  355  873    7  763 9644  515  751 5310  648 3068    7 2351
    87  113 3388 2218 9644 5631 3780   20  233   35   31 2650  597   12
  2419   20   85 4060    7 1741 1429 2414   99    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0   

In [40]:
label_map = {
    0: 'Creative & Media',
    1: 'Education',
    2: 'Engineering',
    3: 'Finance & Accounting',
    4: 'HR & Administration',
    5: 'Healthcare',
    6: 'Hospitality & Service',
    7: 'IT & Software',
    8: 'Legal & Public Service',
    9: 'Manufacturing & Operations',
    10: 'Other',
    11: 'Sales & Marketing',
    12: 'Science & Agriculture'
}

print(label_map[labels[0]])

Sales & Marketing


In [ ]:
mapping = dict(enumerate(le.classes_))
mapping

In [55]:
model.save("model.h5")

#### TFIDF

In [ ]:
X_tfidf = tfidf.fit_transform(df_balanced["job_description"].astype(str))

In [ ]:
X_tfidf_dense = X_tfidf.toarray()
X = X_tfidf_dense
print("TF-IDF shape:", X.shape)

In [ ]:
y = df_balanced["job_function_group"].fillna("unknown")
print("Final X shape:", X.shape)

In [ ]:
class_counts = y.value_counts()
print(class_counts)
small_classes = class_counts[class_counts < 10]

small_classes

In [ ]:
mapping = dict(enumerate(le.classes_))
mapping

In [ ]:
# merge_map = {
#     12: 2,  # Science & Agriculture → Engineering
#     8:  4,  # Legal & Public Service → HR & Administration
#     5:  6   # Healthcare → Hospitality & Service
# }

# y = y.replace(merge_map)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Train the Random Forest model
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Evaluate the model
accuracy_before = rf.score(X_test, y_test)
print(f'Accuracy before feature selection: {accuracy_before:.2f}')

In [ ]:
tfidf_feature_names = tfidf.get_feature_names_out()

feature_names = list(tfidf_feature_names)
feature_importance_df = pd.DataFrame({
    "Feature": feature_names,
    "Importance": rf.feature_importances_
}).sort_values("Importance", ascending=False)

In [ ]:
print(feature_importance_df)

top_feature_indices = feature_importance_df.index[:10].values

X_train_selected = X_train[:, top_feature_indices]
X_test_selected = X_test[:, top_feature_indices]

In [ ]:
# Train the Random Forest model with selected features
rf_selected = RandomForestClassifier(n_estimators=100, random_state=42)
rf_selected.fit(X_train_selected, y_train)

# Evaluate the model
accuracy_after = rf_selected.score(X_test_selected, y_test)
print(f'Accuracy after feature selection: {accuracy_after:.2f}')

In [ ]:
clf = LogisticRegression(
    max_iter=3000,
    n_jobs=-1,
    class_weight="balanced",
    solver="lbfgs"
)

clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
model = LinearSVC(C=1.0)
model.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

In [ ]:
for C in [0.1, 0.5, 1, 2]:
    model = LinearSVC(C=C, class_weight="balanced")
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    print(C)
    print(classification_report(y_test, preds, output_dict=True)["macro avg"]["f1-score"])